In [31]:
import numpy as np
import pandas as pd
import json
import neurokit2 as nk

import sys
sys.path.insert(0, "../code")
import transfer
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib widget

In [2]:
participant = 'GHIVA02_2'

In [3]:
fname = f"../data/{participant}/LOG00005.TXT"

In [4]:
with open(fname, "r") as file:
    for i in range(2):
        line = file.readline()
header = json.loads(line[1:])
meta = header[next(iter(header))]
meta

{'sensor': ['RAW', 'RAW', 'RAW', 'RAW', 'RAW', 'RAW'],
 'device name': 'xx:xx:xx:xx:xx:xx',
 'column': ['nSeq',
  'I1',
  'I2',
  'O1',
  'O2',
  'A1',
  'A2',
  'A3',
  'A4',
  'A5',
  'A6'],
 'sync interval': 2,
 'time': '19:38:09.464',
 'comments': '',
 'device connection': 'OpenLog',
 'channels': [1, 2, 3, 4, 5, 6],
 'date': '2022-04-19',
 'mode': 0,
 'digital IO': [0, 0, 1, 1],
 'firmware version': 52,
 'device': 'bitalino_rev',
 'position': 0,
 'sampling rate': 1000,
 'label': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6'],
 'resolution': [4, 1, 1, 1, 1, 10, 10, 10, 10, 6, 6],
 'special': ''}

In [17]:
srate = meta['sampling rate']

In [5]:
data = np.loadtxt(fname).T

In [10]:
data

array([[  0.,   1.,   2., ...,   5.,   6.,   7.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [253., 252., 252., ..., 652., 653., 652.],
       [ 30.,  30.,  30., ...,  45.,  45.,  45.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [6]:
#data = data[:,30000:data.shape[1]-30000] # skip the first and last 30 seconds (noise)

In [38]:
eegL = data[5] # A1
eegR = data[6] # A2
ecg  = data[7] # A3
eda  = data[8] # A4
ppg  = data[9] # A5

In [39]:
eegL

array([377., 409., 448., ..., 444., 451., 462.])

In [44]:
eegL = transfer.EEG(eegL, meta["resolution"][5])
eegR = transfer.EEG(eegL, meta["resolution"][6])
ecg  = transfer.ECG(ecg, meta["resolution"][7])
eda  = transfer.EDA(ecg, meta["resolution"][8])

In [18]:
time = np.arange(0, eegL.size/srate, 1/srate)

In [14]:
signals, info = nk.bio_process(ecg=ecg, ppg=ppg, eda=eda, sampling_rate=meta["sampling rate"])
results = nk.bio_analyze(signals, sampling_rate=meta["sampling rate"])

/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/neurokit2/epochs/epochs_create.py:128: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(buffer, ignore_index=True, sort=False)
/home/valentin.ghibaudo/.virtualenvs/py38_cluster/lib64/python3.8/site-packages/neurokit2/epochs/epochs_create.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = buffer.append(data, ignore_index=True, sort=False)


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
results

In [ ]:
nk.eda_plot(signals)

In [ ]:
nk.ecg_plot(signals)

In [ ]:
nk.ppg_plot(signals)

In [ ]:
# ECG sitting
signals["ECG_Clean"][10000:20000].plot()

In [ ]:
# ECG walking
signals["ECG_Clean"][400000:410000].plot()

In [ ]:
# PPG sitting
signals["PPG_Clean"][10000:20000].plot()

In [ ]:
# PPG walking
signals["PPG_Clean"][400000:410000].plot()

In [ ]:
# Raw EEG sitting
pd.DataFrame(eegL[10000:11000]).plot()

In [ ]:
# Raw EEG walking
pd.DataFrame(eegL[400000:401000]).plot()